# CV5 - Spark - RDDs

In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("Cv5").getOrCreate()

print(spark)

23/12/01 10:49:05 WARN Utils: Your hostname, gentuwu resolves to a loopback address: 127.0.0.1; using 147.230.1.249 instead (on interface wlo1)
23/12/01 10:49:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/01 10:49:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/01 10:49:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# to stop the session
spark.stop()

## AVG friend Count by age

In [2]:
# read the file located at ./data/fakefriends-header.csv 
# the file is located locally on the machine not on HDFS

# read the file
# Specify the path to your CSV file
csv_file_path = "./data/fakefriends-header.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()


+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     0|    Will| 33|    385|
|     1|Jean-Luc| 26|      2|
|     2|    Hugh| 55|    221|
|     3|  Deanna| 40|    465|
|     4|   Quark| 68|     21|
|     5|  Weyoun| 59|    318|
|     6|  Gowron| 37|    220|
|     7|    Will| 54|    307|
|     8|  Jadzia| 38|    380|
|     9|    Hugh| 27|    181|
|    10|     Odo| 53|    191|
|    11|     Ben| 57|    372|
|    12|   Keiko| 54|    253|
|    13|Jean-Luc| 56|    444|
|    14|    Hugh| 43|     49|
|    15|     Rom| 36|     49|
|    16|  Weyoun| 22|    323|
|    17|     Odo| 35|     13|
|    18|Jean-Luc| 45|    455|
|    19|  Geordi| 60|    246|
+------+--------+---+-------+


In [3]:
# learn the average number of friends by age
# group by age and compute the average number of friends
# round it to 2 decimal places and renames the columns as age and avg_friends

from pyspark.sql.functions import round

result_df = df.groupBy("age").avg("friends").select(
    df["age"].alias("age"),
    round("avg(friends)", 2).alias("avg_friends")
)

result_df.show()


+---+-----------+
|age|avg_friends|
+---+-----------+
| 31|     267.25|
| 65|      298.2|
| 53|     222.86|
| 34|      245.5|
| 28|      209.1|
| 26|     242.06|
| 27|     228.13|
| 44|     282.17|
| 22|     206.43|
| 47|     233.22|
| 52|     340.64|
| 40|     250.82|
| 20|      165.0|
| 57|     258.83|
| 54|     278.08|
| 48|      281.4|
| 19|     213.27|
| 64|     281.33|
| 41|     268.56|
| 43|     230.57|
+---+-----------+


## AVG count of orders by age for each customer

In [4]:
# read the file located at ./data/customer-orders.csv
# the file is located locally on the machine not on HDFS

# read the file
# Specify the path to your CSV file
csv_file_path = "./data/customer-orders.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(csv_file_path, header=False, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()

+---+----+-----+
|_c0| _c1|  _c2|
+---+----+-----+
| 44|8602|37.19|
| 35|5368|65.89|
|  2|3391|40.64|
| 47|6694|14.98|
| 29| 680|13.08|
| 91|8900|24.59|
| 70|3959|68.68|
| 85|1733|28.53|
| 53|9900|83.55|
| 14|1505| 4.32|
| 51|3378| 19.8|
| 42|6926|57.77|
|  2|4424|55.77|
| 79|9291|33.17|
| 50|3901|23.57|
| 20|6633| 6.49|
| 15|6148|65.53|
| 44|8331|99.19|
|  5|3505|64.18|
| 48|5539|32.42|
+---+----+-----+


In [5]:
# learn the total value, 3rd column, spent by each customer
# group by customer id, 1st column, and compute the sum of the 3rd column
# round it to 2 decimal places and renames the columns as customer and total_spent

from pyspark.sql.functions import round

# Assuming "_c0" is the column you want to rename as "customer" and "_c2" as "total_spent"
result_df = df.groupBy("_c0").sum("_c2").select(
    df["_c0"].alias("customer"),
    round("sum(_c2)", 2).alias("total_spent")
).orderBy("total_spent", ascending=False)

result_df.show()

# CHYBI NAZVY SLOUPCU v .csv souboru

+--------+-----------+
|customer|total_spent|
+--------+-----------+
|      68|    6375.45|
|      73|     6206.2|
|      39|    6193.11|
|      54|    6065.39|
|      71|    5995.66|
|       2|    5994.59|
|      97|    5977.19|
|      46|    5963.11|
|      42|    5696.84|
|      59|    5642.89|
|      41|    5637.62|
|       0|    5524.95|
|       8|    5517.24|
|      85|    5503.43|
|      61|    5497.48|
|      32|    5496.05|
|      58|    5437.73|
|      63|    5415.15|
|      15|    5413.51|
|       6|    5397.88|
+--------+-----------+


## SUPERHEROES

- marvel-names.txt neni utf-8

In [6]:
# load the file located at ./data/marvel-names.txt
# the file is located locally on the machine not on HDFS

from pyspark.sql.functions import col, split
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Specify the path to your text file
txt_file_path = "./data/marvel-names.txt"

schema = StructType([ \
    StructField("id", IntegerType(), True), \
    StructField("name", StringType(), True)])

names = spark.read.schema(schema).option("sep", " ").csv(txt_file_path)

names.show()


+---+--------------------+
| id|                name|
+---+--------------------+
|  1|24-HOUR MAN/EMMANUEL|
|  2|3-D MAN/CHARLES CHAN|
|  3|    4-D MAN/MERCURIO|
|  4|             8-BALL/|
|  5|                   A|
|  6|               A'YIN|
|  7|        ABBOTT, JACK|
|  8|             ABCISSA|
|  9|                ABEL|
| 10|ABOMINATION/EMIL BLO|
| 11|ABOMINATION | MUTANT|
| 12|         ABOMINATRIX|
| 13|             ABRAXAS|
| 14|          ADAM 3,031|
| 15|             ABSALOM|
| 16|ABSORBING MAN/CARL C|
| 17|ABSORBING MAN | MUTA|
| 18|                ACBA|
| 19|ACHEBE, REVEREND DOC|
| 20|            ACHILLES|
+---+--------------------+


In [7]:
# load the file located at ./data/marvel-graph.txt
# the file is located locally on the machine not on HDFS
# the first number in the line represents the superhero id and the rest of the numbers in the line are the superhero ids of the superheroes the first superhero has appeared with

from pyspark.sql.functions import split
from pyspark.sql import functions as func

# read the file
# Specify the path to your text file
txt_file_path = "./data/marvel-graph.txt"

lines = spark.read.text(txt_file_path)

connections = lines.withColumn("id", func.split(func.trim(func.col("value")), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.trim(func.col("value")), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))

connections.show()


+----+-----------+
|  id|connections|
+----+-----------+
| 691|          6|
|1159|         11|
|3959|        142|
|1572|         35|
|2294|         14|
|1090|          4|
|3606|        171|
|3414|          7|
| 296|         17|
|4821|         16|
|2162|         41|
|1436|          9|
|1512|         11|
|6194|         14|
|6240|         11|
| 829|         37|
|2136|          6|
|5645|         20|
|2069|        263|
| 467|          0|
+----+-----------+


In [73]:
mostPopular = connections.sort(func.col("connections").desc()).first()

# print the most popular superhero his name and the number of connections\

mostPopularName = names.filter(func.col("id") == mostPopular[0]).select("name").first()

print(mostPopularName[0] + " is the most popular superhero with " + str(mostPopular[1]) + " co-appearances.")

CAPTAIN AMERICA is the most popular superhero with 1933 co-appearances.


In [8]:
# find the top 10 superheroes with the least co-appearances 
# print their names and the number of co-appearances
# sort the result by the number of co-appearances in ascending order
# the coo-appearances cannot be 0

leastPopular = connections.sort(func.col("connections").asc()).filter(func.col("connections") > 0).limit(10)

leastPopularNames = names.join(leastPopular, "id").select("name", "connections")

leastPopularNames.show()

    

+--------------------+-----------+
|                name|connections|
+--------------------+-----------+
|                GOOM|          1|
|HOUSTON, LT. COMMAND|          1|
|         MARAUDER II|          1|
| MASTER OF VENGEANCE|          1|
|               MOTH/|          1|
|              NIMROD|          1|
|   PROFESSOR GIBBON/|          1|
|                SAJA|          1|
|             SLAYER/|          1|
|SPARROW BEAR, MELLIS|          1|
+--------------------+-----------+
